In [1]:
from loader_ref import ExcelLoader

In [3]:
from language_ref import Language

In [2]:
loader=ExcelLoader("/workspaces/de.uke.iam.automapping/src/automapping/VM_Fragebögen_HCHS_10000_20210708.xlsx", 'Kurzname', 'Langname')

In [4]:
sample = loader.load(Language.GERMAN)[0]

In [7]:
from translator_ref import HuggingFace

: 

: 